In [ ]:
import timeit

In [ ]:
import pandas as pd

import warnings
warnings.simplefilter(action='ignore')



from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters

# import dask.dataframe as dd

from dask.distributed import Client
client = Client()

# from tsfresh.convenience.bindings import dask_feature_extraction_on_chunk

In [ ]:
client

In [ ]:
CO_timeseries_unpivot_path = '/home/ubuntu/workspace_rohan/project/data/interim/CO_timeseries_unpivot_shared/_0_400000.csv'


In [ ]:
# CO_timeseries_featurized = '/home/ubuntu/workspace_rohan/project/data/processed/CO_timeseries_featurized_shared/CO_timeseries_featurized_0_400000'

In [ ]:
# df = dd.read_csv(CO_timeseries_unpivot_path)

pandas sharding

In [ ]:
# CO_timeseries_shared_prepath = '/home/ubuntu/workspace_rohan/project/data/interim/CO_timeseries_unpivot_shared/'

In [ ]:
df_pd = pd.read_csv(CO_timeseries_unpivot_path)

In [ ]:
# df_pd = df_pd[['index', 'week_int', 'week_value']]

In [ ]:
# start = 0
# while (start < round(df_pd.shape[0]/19)):
#     end = start+400000
#     print(start, end)
#     df_temp = df_pd[(df_pd['index']>=start) & (df_pd['index']<end)]
#     df_temp.to_csv(CO_timeseries_shared_prepath+'_'+str(start)+'_'+str(end)+'.csv', index=False)
#     start = start+400000

In [ ]:
df = df[['index', 'week_int', 'week_value']]

In [ ]:
df.head(2)

In [ ]:
# each = 787 features

In [ ]:
# X = extract_features(df,
#                      column_id="index", column_sort="week_int",
#                      pivot=False)

In [ ]:
# X.compute()

In [ ]:
df = df.melt(id_vars=["index", "week_int"],
               value_vars=["week_value"],
               var_name="kind", value_name="value")

In [ ]:
df_grouped = df.groupby(["index", "kind"])

In [ ]:
features = dask_feature_extraction_on_chunk(df_grouped,
                                            column_id="index",
                                            column_kind="kind",
                                            column_sort="week_int",
                                            column_value="value")

In [ ]:
features = features.categorize(columns=["variable"])

In [ ]:
features = features.reset_index(drop=True).pivot_table(index="index", columns="variable", values="value", aggfunc="mean")

In [ ]:
features.to_csv(CO_timeseries_featurized)

Robot Example

In [ ]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, \
    load_robot_execution_failures
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

In [ ]:
timeseries.head()

In [ ]:
timeseries.to_csv("robot.csv", index=False)

In [ ]:
timeseries = dd.read_csv("robot.csv")

In [ ]:
timeseries

In [ ]:
timeseries = timeseries[(timeseries['id']<=1000)]

In [ ]:
timeseries = timeseries[['id', 'time', 'F_x']]

In [ ]:
timeseries

In [ ]:
df = timeseries.melt(id_vars=["id", "time"],
               value_vars=["F_x"],
               var_name="kind", value_name="value")

In [ ]:
df

In [ ]:
df_grouped = df.groupby(["id", "kind"])

In [ ]:
df_grouped

In [ ]:
from tsfresh.feature_extraction.settings import MinimalFCParameters

In [ ]:
features = dask_feature_extraction_on_chunk(df_grouped,
                                            column_id="id",
                                            column_kind="kind",
                                            column_sort="time",
                                            column_value="value")

In [ ]:
features = features.categorize(columns=["variable"])

In [ ]:
features = features.reset_index(drop=True).pivot_table(index="id", columns="variable", values="value", aggfunc="mean")

In [ ]:
features

In [ ]:
features.to_csv("robot_feat")

In [ ]:
s = pd.read_csv("/home/ubuntu/workspace_rohan/project/notebooks/robot_feat/0.part")

parallel dash local

In [ ]:
from tsfresh.feature_extraction import extract_features
from tsfresh.utilities.distribution import LocalDaskDistributor

In [ ]:
Distributor = LocalDaskDistributor(n_workers=4)

In [ ]:
Distributor

In [ ]:
X = extract_features(timeseries_container=df_pd,
                     column_id='index', column_sort='week_int',
                     distributor=Distributor)

In [ ]:
from tsfresh.utilities.distribution import ClusterDaskDistributor

In [ ]:
# Distributor = ClusterDaskDistributor(address="127.0.0.1:33377")

# X = extract_features(timeseries_container=df_pd,
#                      column_id='index', column_sort='week_int',
#                      distributor=Distributor)

In [ ]:
df_pd.shape[0]/19

In [ ]:
Distributor = ClusterDaskDistributor(address="127.0.0.1:33377")

In [ ]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
start = 0
while (start < round(df_pd.shape[0]/19)):
    end = start+4000
#     print(start, end)
    df_temp = df_pd[(df_pd['index']>=start) & (df_pd['index']<end)]
    
    

    X = extract_features(timeseries_container=df_temp,
                         column_id='index', column_sort='week_int',
                         disable_progressbar=False,
                         distributor=Distributor)
    
#     df_temp.to_csv(CO_timeseries_shared_prepath+'_'+str(start)+'_'+str(end)+'.csv', index=False)
    start = start+4000
    break

print("Total training time is :", timeit.default_timer() - starttime)

In [ ]:
73

In [ ]:
X.shape

In [ ]:
10*101*73